# Regression Week 2: Multiple Regression (Interpretation) using Scikit Learn

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [2]:
sales = pd.read_csv('kc_house_data.csv')
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [4]:
y = sales['price']
X = sales.loc[:, sales.columns != 'price']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
X_train1 = X_train[example_features]
X_test1 = X_test[example_features]

In [7]:
mr1 = LinearRegression().fit(X_train1, y_train)

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [8]:
#Accuracy is a classification metric. You can't use it with a regression. See the documentation for info on the various metrics.
#from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [9]:
y_predict_train1 = mr1.predict(X_train1)
y_predict_test1 = mr1.predict(X_test1)

In [13]:
print("mean sqaured error of training: {}".format(mean_squared_error(y_train, y_predict_train1)))
print("mean sqaured error of testing: {}".format(mean_squared_error(y_test, y_predict_test1)))
print("R2 of training: {}".format(r2_score(y_train, y_predict_train1)))
print("R2 of testing: {}".format(r2_score(y_test, y_predict_test1)))

mean sqaured error of training: 68100328412.36793
mean sqaured error of testing: 59961501759.44609
R2 of training: 0.5090198210347909
R2 of testing: 0.49580096548774155


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [22]:
from math import log
import numpy as np

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [24]:
X_train['bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x**2)
X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)

X_train['bed_bath_rooms'] = X_train['bedrooms'] * X_train['bathrooms']
X_test['bed_bath_rooms'] = X_test['bedrooms'] * X_test['bathrooms']

X_train['log_sqft_living'] = np.log(X_train['sqft_living'])
X_test['log_sqft_living'] = np.log(X_test['sqft_living'])

X_train['lat_plus_long'] = X_train['lat'] + X_train['long']
X_test['lat_plus_long'] = X_test['lat'] + X_test['long']

C:\Users\vanch\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\vanch\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vanch\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)


In [25]:
X_train

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
5268,5100402668,20150218T000000,3,1.00,1570,5510,1.0,0,0,4,...,0,98115,47.6942,-122.319,1770,6380,9,3.00,7.358831,-74.6248
16909,7856560480,20140808T000000,3,2.50,1780,11000,1.0,0,0,4,...,0,98006,47.5574,-122.149,2310,9700,9,7.50,7.484369,-74.5916
16123,2872900010,20150414T000000,3,1.50,1090,9862,1.0,0,0,3,...,0,98074,47.6256,-122.036,1710,9862,9,4.50,6.993933,-74.4104
12181,3216900070,20140617T000000,4,2.50,2210,7079,2.0,0,0,3,...,0,98031,47.4206,-122.183,1970,7000,16,10.00,7.700748,-74.7624
12617,976000790,20141020T000000,3,2.50,1800,4763,2.0,0,0,3,...,0,98119,47.6460,-122.362,1790,4763,9,7.50,7.495542,-74.7160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,2322069010,20141007T000000,5,5.00,3960,94089,2.0,0,0,3,...,0,98038,47.3800,-122.011,2240,64468,25,25.00,8.283999,-74.6310
19648,2114700368,20141118T000000,2,2.50,1400,1262,2.0,0,0,3,...,0,98106,47.5342,-122.349,1060,1524,4,5.00,7.244228,-74.8148
9845,5469501200,20140820T000000,3,2.25,2360,14950,1.0,0,0,4,...,0,98042,47.3856,-122.158,2720,14388,9,6.75,7.766417,-74.7724
10799,3751602797,20140702T000000,4,2.00,2370,76665,2.0,0,0,4,...,0,98001,47.2831,-122.279,2110,19334,16,8.00,7.770645,-74.9959


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [26]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

## Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude

In [27]:
X_train_M1 = X_train[model_1_features]
X_test_M1 = X_test[model_1_features]

In [28]:
mrModel1 = LinearRegression().fit(X_train_M1, y_train)

In [29]:
y_predict_train_M1 = mrModel1.predict(X_train_M1)
y_predict_test_M1 = mrModel1.predict(X_test_M1)

In [30]:
print("mean sqaured error of training: {}".format(mean_squared_error(y_train, y_predict_train_M1)))
print("mean sqaured error of testing: {}".format(mean_squared_error(y_test, y_predict_test_M1)))
print("R2 of training: {}".format(r2_score(y_train, y_predict_train_M1)))
print("R2 of testing: {}".format(r2_score(y_test, y_predict_test_M1)))

mean sqaured error of training: 56671116112.68534
mean sqaured error of testing: 49384022511.93766
R2 of training: 0.5914205499468472
R2 of testing: 0.5847439483630335


# Model 2: add bedrooms*bathrooms

In [31]:
X_train_M2 = X_train[model_2_features]
X_test_M2 = X_test[model_2_features]

In [32]:
mrModel2 = LinearRegression().fit(X_train_M2, y_train)

In [33]:
y_predict_train_M2 = mrModel2.predict(X_train_M2)
y_predict_test_M2 = mrModel2.predict(X_test_M2)

In [34]:
print("mean sqaured error of training: {}".format(mean_squared_error(y_train, y_predict_train_M2)))
print("mean sqaured error of testing: {}".format(mean_squared_error(y_test, y_predict_test_M2)))
print("R2 of training: {}".format(r2_score(y_train, y_predict_train_M2)))
print("R2 of testing: {}".format(r2_score(y_test, y_predict_test_M2)))

mean sqaured error of training: 56148016178.691666
mean sqaured error of testing: 48757470314.35226
R2 of training: 0.5951919223498363
R2 of testing: 0.5900124457125737


# Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [35]:
X_train_M3 = X_train[model_3_features]
X_test_M3 = X_test[model_3_features]

In [40]:
mrModel3 = LinearRegression().fit(X_train_M3, y_train)

In [41]:
y_predict_train_M3 = mrModel3.predict(X_train_M3)
y_predict_test_M3 = mrModel3.predict(X_test_M3)

In [42]:
print("mean sqaured error of training: {}".format(mean_squared_error(y_train, y_predict_train_M3)))
print("mean sqaured error of testing: {}".format(mean_squared_error(y_test, y_predict_test_M3)))
print("R2 of training: {}".format(r2_score(y_train, y_predict_train_M3)))
print("R2 of testing: {}".format(r2_score(y_test, y_predict_test_M3)))

mean sqaured error of training: 52842894446.20923
mean sqaured error of testing: 47182986795.65516
R2 of training: 0.619020724611843
R2 of testing: 0.6032518250924852
